## О задании
В этом задании вы будете предсказывать один из параметров человека (рост, вес, возраст).

Работать мы будем с датасетом человеческих параметров [Body Fat Prediction Dataset](https://www.kaggle.com/datasets/fedesoriano/body-fat-prediction-dataset).

### Формат сдачи
Данное задание сдаётся через [эту гугл форму](https://forms.gle/YdvLRGWAsNaNvQvs7). Не забудьте открыть доступ по ссылке к файлу (справа сверху "Поделиться" или "Share")

## Приступим

Сделаем импорты

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Скачаем данные

In [ ]:
!gdown --id 1mgembIyezFrbTlNlRFO5fZu7TcBfT_Uu

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1mgembIyezFrbTlNlRFO5fZu7TcBfT_Uu
To: /content/bodyfat.csv
100% 19.6k/19.6k [00:00<00:00, 26.7MB/s]


Загрузите данные из файла `bodyfat.csv` и сохраните в переменную `df`. Выведите первые 5 строк, число строк и столбцов

In [ ]:
### ЗДЕСЬ ВАШ КОД ¯\_(ツ)_/¯ 

df = None

### Первичный анализ

Ознакомиться со значениями признаков вы можете [по ссылке](https://www.kaggle.com/datasets/fedesoriano/body-fat-prediction-dataset)

Посмотрите, есть ли пропуски в данных. Если есть, то избавьтесь от них. Вам может понадобиться метод [`dropna`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html)

In [ ]:
### ЗДЕСЬ ВАШ КОД ¯\_(ツ)_/¯ 

Построим карту корреляций

In [ ]:
df.corr().style.background_gradient('gray').format('{:.2g}'.format)

,Density,BodyFat,Age,Weight,Height,Neck,Chest,Abdomen,Hip,Thigh,Knee,Ankle,Biceps,Forearm,Wrist
Density,1,-0.99,-0.28,-0.59,0.098,-0.47,-0.68,-0.8,-0.61,-0.55,-0.5,-0.26,-0.49,-0.35,-0.33
BodyFat,-0.99,1,0.29,0.61,-0.089,0.49,0.7,0.81,0.63,0.56,0.51,0.27,0.49,0.36,0.35
Age,-0.28,0.29,1,-0.013,-0.17,0.11,0.18,0.23,-0.05,-0.2,0.018,-0.11,-0.041,-0.085,0.21
Weight,-0.59,0.61,-0.013,1,0.31,0.83,0.89,0.89,0.94,0.87,0.85,0.61,0.8,0.63,0.73
Height,0.098,-0.089,-0.17,0.31,1,0.25,0.13,0.088,0.17,0.15,0.29,0.26,0.21,0.23,0.32
Neck,-0.47,0.49,0.11,0.83,0.25,1,0.78,0.75,0.73,0.7,0.67,0.48,0.73,0.62,0.74
Chest,-0.68,0.7,0.18,0.89,0.13,0.78,1,0.92,0.83,0.73,0.72,0.48,0.73,0.58,0.66
Abdomen,-0.8,0.81,0.23,0.89,0.088,0.75,0.92,1,0.87,0.77,0.74,0.45,0.68,0.5,0.62
Hip,-0.61,0.63,-0.05,0.94,0.17,0.73,0.83,0.87,1,0.9,0.82,0.56,0.74,0.55,0.63
Thigh,-0.55,0.56,-0.2,0.87,0.15,0.7,0.73,0.77,0.9,1,0.8,0.54,0.76,0.57,0.56


**Вопрос.** Глядя на карту корреляций, можете ли вы выявить линейно зависимые признаки?

<u>**Ответ**</u>: _ _ _ _ _ _ _ _ _ _ _ _

### Определение целевой и объясняющих переменных

Выберите величину, которую вы собираетесь предсказывать. На выбор: **возраст**, **рост** или **вес**. Укажите это снизу:

### Я предсказываю <u>**возраст**</u>

Отберите признаки, по которым вы будете предсказывать целевую переменную (список из названий столбцов) и запишите их в переменную `cols`.

**Вопрос.** По какому критерию нужно отбирать объясняющие переменные (переменные, по которым предсказываем целевую)

<u>**Ответ**</u>: _ _ _ _ _ _ _ _ _ _ _ _

**(*) Дополнительно.** Постройте графики рассеяния между объясняющими переменными и целевой переменной. Наблюдаются ли линейные зависимости?

<u>**Ответ**</u>: _ _ _ _ _ _ _ _ _ _ _ _

In [ ]:
### ЗДЕСЬ ВАШ КОД ¯\_(ツ)_/¯ 

cols = None # список из столбцов - объясняющих переменных
X = df[cols] # отбираем нужные столбцы из исходного датафрейма

Запишите в переменную `y` столбец-признак, который предсказываете, а в переменную `X` признаки, по которым предсказываете

In [ ]:
### ЗДЕСЬ ВАШ КОД ¯\_(ツ)_/¯ 

X, y = None, None

### Разбиение данных

Разбейте данные на тренировочную и тестовую части. На тестовую выделите 20% выборки, можно зафиксировать `random_state`

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
### ЗДЕСЬ ВАШ КОД ¯\_(ツ)_/¯ 

X_train, X_test, y_train, y_test = None

### Масштабирование признаков

Отмасштабируйте признаки с помощью [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html). Не забудьте, что scaler "обучается" только на тренировочных признаках

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
### ЗДЕСЬ ВАШ КОД ¯\_(ツ)_/¯ 

scaler = StandardScaler()

### Построение модели линейной регрессии

Обучите модель [`Ridge`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html) на тренировочных данных и сделайте предсказание

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
### ЗДЕСЬ ВАШ КОД ¯\_(ツ)_/¯

linreg = None # здесь будет ваша модель
y_pred_train = None # здесь будут предсказания на тренировочных данных
y_pred_test = None # здесь будут предсказания на тестовых данных

В качестве метрики возьмём **RMSE**. Оцените качество по этой метрике на тренировочной и тестовой выборках. Для вычисления метрики можете воспользоваться функцией **rmse_score**

In [ ]:
def rmse_score(y_true, y_pred):
    error = (y_true - y_pred) ** 2
    return np.sqrt(np.mean(error))

In [ ]:
### ЗДЕСЬ ВАШ КОД ¯\_(ツ)_/¯

print('Качество на тренировочной выборке:', None)
print('Качество на тестовой выборке:', None)

Качество на тренировочной выборке: None
Качество на тестовой выборке: None


**(*) Дополнительно.** Какое качество будет у константной модели по выбранной метрике? Лучше ли предсказания нашей модели?

<u>**Ответ**</u>: _ _ _ _ _ _ _ _ _ _ _ _

### Подбор гиперпараметров

Попробуйте подобрать лучший коэффициент регуляризации с помощью кросс-валидации. Для этого можно воспользоваться готовым классом [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html)

Основные моменты:
* Запишите список значений коэффициента регуляризацию в переменную `alphas`
* Не забудьте указать параметр `scoring` равным "neg_root_mean_squared_error"

In [ ]:
from sklearn.linear_model import RidgeCV

In [ ]:
### ЗДЕСЬ ВАШ КОД ¯\_(ツ)_/¯

alphas = [0.1, 1, 10] # можно добавить другие перебираемые значения
ridge_cv = None # модель со встроенной кросс-валидацией

**Вопрос.** Какое получилось лучшее значений гиперпараметра?

In [ ]:
### ЗДЕСЬ ВАШ КОД ¯\_(ツ)_/¯

print('Лучшее значение гиперпараметра:', None)

Обучите модель `Ridge` с лучшим значением гиперпараметра на тренировочных данных и оцените качество модели

In [ ]:
### ЗДЕСЬ ВАШ КОД ¯\_(ツ)_/¯

linreg = None # здесь модель с лучшим значением коэффициента регуляризации
print('Качество на тренировочной выборке:', None)
print('Качество на тестовой выборке:', None)

**Вопрос.** Улучшилось ли качество модели после подбора гиперпараметра?

<u>**Ответ**</u>: _ _ _ _ _ _ _ _ _ _ _ _

**(*) Дополнительно.** Можете поварьировать число сплитов (параметр `cv`). Отразилось ли это на лучшем значении гиперпараметра/качестве модели?

<u>**Ответ**</u>: _ _ _ _ _ _ _ _ _ _ _ _

### Выводы


**Вопрос.** Напишите выводы о проделанной работе: что было сделано, какие результаты получены?

<u>**Ответ**</u>: _ _ _ _ _ _ _ _ _ _ _ _

### (*) Дополнительно

**(*) Дополнительно.** Обучите модель [`Lasso`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html), попробуйте также подобрать коэффициенты регуляризации (вам может пригодиться [`LassoCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html)) и оценить модель с лучшим гиперпараметром. Сравните получившиеся модели Ridge и Lasso

<u>**Ответ**</u>: _ _ _ _ _ _ _ _ _ _ _ _

**(*) Дополнительно** Для улучшения качества модели может помочь более качественная предобработка данных. Попробуйте:
1. Вместо выбрасывания строк с пустыми значениями попробуйте заполнить их средним (или медианой) по столбцу. Помните, что среднее считается только по тренировочной выборке
2. Попробуйте выявить выбросы в данных. Избавьтесь от них или заполните адекватным значением
3. Попробуйте другой способ масштабирования (например, [`MinMaxScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)) или не масштабировать данные
4. Посмотрите на распределение целевой переменной. Попробуйте привести его к нормальному распределению (логарифмирование целевой переменной, возведение в степень,..). При подсчёте метрики придётся применить обратное преобразование целевой переменной

Напишите результаты экспериментов:

<u>**Ответ**</u>: _ _ _ _ _ _ _ _ _ _ _ _